In [ ]:
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [ ]:
model_id = "BSC-LT/salamandra-7b-instruct-aina-hack"

# Open the .data file and read the content
file_path = './data/output_.data'

# Open and read the file content
with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# General prompt to summarize the content
text = f"""
Please summarize the key points of the following document. Focus on the main topics, headings, and important information. Exclude unnecessary details like references, footnotes, and additional metadata. If the document contains multiple sections, provide a brief summary of each section.

Document:
{file_content}
"""

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
  )

message = [ { "role": "user", "content": text } ]
date_string = datetime.today().strftime('%Y-%m-%d')

prompt = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True,
    date_string=date_string
)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=200)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))